# Lab 03 · Gemini Proxy Chat

*This lab notebook provides guided steps. All commands are intended for local execution.*

## Objectives
- A Gemini proxy endpoint is created in FastAPI.
- React chat UI components are connected to the proxy.
- API keys remain confined to the backend.

## What will be learned
- Backend proxy patterns for hosted LLMs are practiced.
- Basic chat state management in React is reviewed.
- Environment variable usage is reinforced.

## Prerequisites & install
The following commands are intended for local execution.

```bash
cd ai-web/backend
. .venv/bin/activate
pip install google-generativeai
```

## Step-by-step tasks
### Step 1: Gemini helper module
A backend helper is written so Gemini calls are centralized.

In [ ]:
from pathlib import Path
module = Path("ai-web/backend/app/llm.py")
module.parent.mkdir(parents=True, exist_ok=True)
module.write_text('''import os
from typing import Dict, List, Any

from google import genai


def chat(messages: List[Dict[str, Any]]) -> str:
  api_key = os.environ.get('GEMINI_API_KEY', '')
  if not api_key:
    raise RuntimeError('A backend API key is required.')
  client = genai.Client(api_key=api_key)
  response = client.models.generate_content(
      model="gemini-1.5-flash",
      contents=messages,
  )
  return response.text
''')
print("Gemini helper was written.")

### Step 2: FastAPI route update
The FastAPI app is expanded with /api/chat.

In [ ]:
from pathlib import Path
main_path = Path("ai-web/backend/app/main.py")
text = main_path.read_text()
addition = '''
from typing import List
from pydantic import BaseModel
from .llm import chat as gemini_chat


class ChatTurn(BaseModel):
    role: str
    content: str


class ChatRequest(BaseModel):
    messages: List[ChatTurn]


@app.post("/api/chat")
def chat_endpoint(request: ChatRequest):
    transcript = [
        {"role": turn.role, "parts": [turn.content]} for turn in request.messages
    ]
    text = gemini_chat(transcript)
    return {"text": text}
'''
if "chat_endpoint" not in text:
    main_path.write_text(text.rstrip() + "
" + addition)
    print("FastAPI route was appended.")
else:
    print("FastAPI route already present.")

### Step 3: React chat surface
A lightweight chat component is appended to App.jsx so the proxy is exercised.

In [ ]:
from pathlib import Path
app_js = Path("ai-web/frontend/src/App.jsx")
app_js.write_text('''import React, { useState } from 'react';
import { post } from './lib/api';
import { withRetry } from './lib/retry';

function App() {
  const [messages, setMessages] = useState([{ role: 'user', content: 'Hello Gemini' }]);
  const [input, setInput] = useState('');
  const [loading, setLoading] = useState(false);
  const [error, setError] = useState('');

  async function handleSend(event) {
    event.preventDefault();
    const updated = [...messages, { role: 'user', content: input }];
    setMessages(updated);
    setLoading(true);
    setError('');
    try {
      const result = await withRetry(
        () => post('/api/chat', { messages: updated }),
        1,
        800
      );
      setMessages([...updated, { role: 'model', content: result.text }]);
      setInput('');
    } catch (err) {
      setError('The proxy was unreachable. Please try again.');
    } finally {
      setLoading(false);
    }
  }

  return (
    <main style={{ padding: 24 }}>
      <h1>Lab 3 — Gemini proxy chat</h1>
      <section>
        {messages.map((msg, index) => (
          <p key={index}>
            <strong>{msg.role}:</strong> {msg.content}
          </p>
        ))}
      </section>
      <form onSubmit={handleSend}>
        <input
          value={input}
          onChange={(event) => setInput(event.target.value)}
          placeholder="Type a follow-up"
        />
        <button type="submit" disabled={loading || !input}>Send</button>
      </form>
      {loading && <p>Awaiting proxy response…</p>}
      {error && <p style={{ color: 'red' }}>{error}</p>}
    </main>
  );
}

export default App;
''')
print("Chat UI was seeded.")

## Validation / acceptance checks
```bash
# locally
curl -X POST http://localhost:8000/api/chat -H 'Content-Type: application/json' -d '{"messages":[{"role":"user","content":"Hello"}]}'
```
- A JSON response with a text field is produced by the backend proxy.
- React development mode shows the described UI state without console errors.

## Homework / extensions
- Streaming responses are researched for future enhancements.
- Chat history persistence is sketched for the next lab.